In [1]:
train_data = {
    '支付问题': [
        "为什么我的支付失败了?",
        "怎么绑定我的银行卡进行支付?",
    ],
    '商品咨询': [
        "这款商品有货吗?",
        "请问这款手机有什么功能?",
    ],
    '促销活动': [
        "现在有什么促销活动吗?",
        "我怎么参加你们的优惠活动?",
    ],
    '发票问题': [
        "我怎么申请电子发票?",
        "发票什么时候可以开出来?",
    ],
    '账户安全': [
        "我的账户被盗了怎么办?",
        "如何开启账户的双重验证?",
    ],
    '售后服务': [
        "你们提供售后服务吗?",
        "保修期内维修是免费的吗?",
    ],
}

In [2]:
class IntentDataset:
    def __init__(self, data):
        self.data = data
        self.intent_labels = list(data.keys())

    def gen_demonstration(self, shot_num):
        demonstrations = []
        for intent in self.intent_labels:
            examples = random.sample(self.data[intent], shot_num)
            demonstrations.append(self._format_demonstration(intent, examples))
        return '\n'.join(demonstrations)

In [5]:
!pip install dspy

In [10]:
!pip install dspy-ai openai

In [13]:
import dspy
import random
import numpy as np
from typing import List
from dspy.teleprompt import MIPROv2
from tqdm import tqdm

import dspy
import random
from typing import Dict, List
from openai import OpenAI

In [21]:
# DSPy意图识别 - 完全修复版本

# 安装依赖
!pip install dspy-ai openai -q

import dspy
import random
from typing import Dict, List

# 配置API
API_KEY = "sk-74c8da577a104fa78356b151d7a5536d"

# 训练数据集
train_data = {
    '支付问题': [
        "为什么我的支付失败了?",
        "怎么绑定我的银行卡进行支付?",
        "支付时显示余额不足怎么办?",
        "可以使用支付宝付款吗?",
        "退款什么时候到账?",
        "支付密码忘记了怎么办?"
    ],
    '商品咨询': [
        "这款商品有货吗?",
        "请问这款手机有什么功能?",
        "这个产品的规格参数是什么?",
        "有其他颜色可以选择吗?",
        "这款商品什么时候上市的?",
        "能介绍一下这个产品的特点吗?"
    ],
    '促销活动': [
        "现在有什么促销活动吗?",
        "我怎么参加你们的优惠活动?",
        "双十一有折扣吗?",
        "新用户有什么优惠?",
        "会员有专属优惠吗?",
        "这个商品有满减活动吗?"
    ],
    '发票问题': [
        "我怎么申请电子发票?",
        "发票什么时候可以开出来?",
        "可以开增值税专用发票吗?",
        "发票抬头可以修改吗?",
        "发票丢失了怎么办?",
        "如何下载电子发票?"
    ],
    '账户安全': [
        "我的账户被盗了怎么办?",
        "如何开启账户的双重验证?",
        "怎么修改登录密码?",
        "账户异常登录怎么处理?",
        "如何绑定手机号码?",
        "账户被锁定了怎么解锁?"
    ],
    '售后服务': [
        "你们提供售后服务吗?",
        "保修期内维修是免费的吗?",
        "商品质量有问题可以退货吗?",
        "如何申请售后服务?",
        "保修期是多长时间?",
        "可以上门维修吗?"
    ],
}

class IntentDataset:
    def __init__(self, data):
        self.data = data
        self.intent_labels = list(data.keys())

    def create_train_examples(self):
        examples = []
        for intent, queries in self.data.items():
            for query in queries:
                # 使用新版DSPy的格式
                example = dspy.Example(
                    query=query,
                    intent=intent
                ).with_inputs('query')
                examples.append(example)
        return examples

    def create_test_examples(self):
        test_queries = [
            ("我上周买的电脑坏了,可以换一台新的吗?", "售后服务"),
            ("怎么使用优惠券?", "促销活动"),
            ("忘记密码了怎么登录?", "账户安全"),
            ("这个商品是正品吗?", "商品咨询"),
            ("微信支付失败了", "支付问题"),
            ("需要开发票", "发票问题"),
            ("我的银行卡绑定失败了", "支付问题"),
            ("这个手机有什么颜色可以选择", "商品咨询"),
            ("双十二有活动吗", "促销活动"),
            ("电子发票怎么下载", "发票问题")
        ]

        examples = []
        for query, intent in test_queries:
            # 使用新版DSPy的格式
            example = dspy.Example(
                query=query,
                intent=intent
            ).with_inputs('query')
            examples.append(example)
        return examples

class IntentClassifier(dspy.Signature):
    """识别用户询问的意图类别"""
    query = dspy.InputField(desc="用户的询问内容")
    intent = dspy.OutputField(desc="意图类别：支付问题、商品咨询、促销活动、发票问题、账户安全、售后服务")

class IntentModel(dspy.Module):
    def __init__(self):
        super().__init__()
        self.classify = dspy.ChainOfThought(IntentClassifier)

    def forward(self, query):
        result = self.classify(query=query)
        return result

def validate_intent(example, pred, trace=None):
    return pred.intent == example.intent

# 主程序
def main():
    # 初始化数据集
    print("=== 初始化数据集 ===")
    dataset = IntentDataset(train_data)
    train_examples = dataset.create_train_examples()
    test_examples = dataset.create_test_examples()

    print(f"训练样本数量: {len(train_examples)}")
    print(f"测试样本数量: {len(test_examples)}")
    print(f"意图类别: {dataset.intent_labels}")

    # 配置DSPy
    print("\n=== 配置API ===")
    if API_KEY.startswith("sk-") and len(API_KEY) > 20:
        try:
            dspy.settings.configure(
                lm=dspy.LM(
                    model="deepseek/deepseek-chat",
                    api_key=API_KEY,
                    max_tokens=100,
                    temperature=0.1
                )
            )
            print("✅ API配置成功！")

            # 初始化模型
            model = IntentModel()

            # 单个测试
            print("\n=== 单个测试 ===")
            test_query = "我上周买的电脑坏了,可以换一台新的吗?"
            pred = model(query=test_query)
            print(f"询问: {test_query}")
            print(f"预测意图: {pred.intent}")

            # 批量测试
            print("\n=== 批量测试结果 ===")
            correct_count = 0
            total_count = len(test_examples)

            for i, example in enumerate(test_examples):
                try:
                    pred = model(query=example.query)
                    is_correct = pred.intent == example.intent
                    if is_correct:
                        correct_count += 1

                    status = "✅" if is_correct else "❌"
                    print(f"{status} 测试 {i+1}:")
                    print(f"   询问: {example.query}")
                    print(f"   预测: {pred.intent}")
                    print(f"   真实: {example.intent}")
                    print("-" * 50)

                except Exception as e:
                    print(f"❌ 测试 {i+1} 失败: {e}")

            accuracy = correct_count / total_count
            print(f"\n📊 准确率: {correct_count}/{total_count} = {accuracy:.2%}")

            # 模型优化
            print("\n=== 模型优化 ===")
            compiled_model = model  # 默认使用原始模型

            try:
                from dspy.teleprompt import BootstrapFewShot
                optimizer = BootstrapFewShot(metric=validate_intent)
                compiled_model = optimizer.compile(model, trainset=train_examples[:15])
                print("✅ 模型优化完成！")
            except Exception as e:
                print(f"⚠️ 模型优化失败: {e}")
                print("使用原始模型继续测试")

            # 优化后测试
            print("\n=== 优化后模型测试 ===")
            optimized_correct = 0

            for i, example in enumerate(test_examples):
                try:
                    pred = compiled_model(query=example.query)
                    is_correct = pred.intent == example.intent
                    if is_correct:
                        optimized_correct += 1

                    status = "✅" if is_correct else "❌"
                    print(f"{status} 优化测试 {i+1}: {example.query} -> {pred.intent}")

                except Exception as e:
                    print(f"❌ 优化测试 {i+1} 失败: {e}")

            optimized_accuracy = optimized_correct / total_count
            print(f"\n📊 优化后准确率: {optimized_correct}/{total_count} = {optimized_accuracy:.2%}")
            print(f"📈 提升: {(optimized_accuracy - accuracy)*100:.1f}%")

            # 自定义测试
            print("\n=== 自定义测试 ===")
            custom_queries = [
                "我的银行卡绑定失败了",
                "这个手机有什么颜色可以选择",
                "双十二有活动吗",
                "电子发票怎么下载",
                "账号密码忘记了",
                "买的东西质量不好想退货",
                "会员积分怎么使用",
                "物流信息在哪里查看"
            ]

            for query in custom_queries:
                try:
                    pred = compiled_model(query=query)
                    print(f"🔍 {query} -> {pred.intent}")
                except Exception as e:
                    print(f"❌ {query} -> 预测失败: {e}")

            print("\n🎉 测试完成！")

        except Exception as e:
            print(f"❌ API配置失败: {e}")
            print("请检查API密钥是否正确")
    else:
        print("❌ 请先设置您的API密钥！")
        print("API密钥应该以 'sk-' 开头")

# 运行主程序
main()

print("\n" + "="*60)
print("📝 使用说明:")
print("1. 确认API密钥格式: sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")
print("2. 直接运行代码查看意图识别效果")
print("3. 可以修改 custom_queries 列表测试更多场景")
print("="*60)

=== 初始化数据集 ===
训练样本数量: 36
测试样本数量: 10
意图类别: ['支付问题', '商品咨询', '促销活动', '发票问题', '账户安全', '售后服务']

=== 配置API ===
✅ API配置成功！

=== 单个测试 ===
询问: 我上周买的电脑坏了,可以换一台新的吗?
预测意图: 售后服务

=== 批量测试结果 ===
✅ 测试 1:
   询问: 我上周买的电脑坏了,可以换一台新的吗?
   预测: 售后服务
   真实: 售后服务
--------------------------------------------------
✅ 测试 2:
   询问: 怎么使用优惠券?
   预测: 促销活动
   真实: 促销活动
--------------------------------------------------
✅ 测试 3:
   询问: 忘记密码了怎么登录?
   预测: 账户安全
   真实: 账户安全
--------------------------------------------------
✅ 测试 4:
   询问: 这个商品是正品吗?
   预测: 商品咨询
   真实: 商品咨询
--------------------------------------------------
✅ 测试 5:
   询问: 微信支付失败了
   预测: 支付问题
   真实: 支付问题
--------------------------------------------------
✅ 测试 6:
   询问: 需要开发票
   预测: 发票问题
   真实: 发票问题
--------------------------------------------------
✅ 测试 7:
   询问: 我的银行卡绑定失败了
   预测: 支付问题
   真实: 支付问题
--------------------------------------------------
✅ 测试 8:
   询问: 这个手机有什么颜色可以选择
   预测: 商品咨询
   真实: 商品咨询
--------------------------------------------------
✅ 测试 9:


 27%|██▋       | 4/15 [00:22<01:02,  5.68s/it]


Bootstrapped 4 full traces after 4 examples for up to 1 rounds, amounting to 4 attempts.
✅ 模型优化完成！

=== 优化后模型测试 ===
✅ 优化测试 1: 我上周买的电脑坏了,可以换一台新的吗? -> 售后服务
✅ 优化测试 2: 怎么使用优惠券? -> 促销活动
✅ 优化测试 3: 忘记密码了怎么登录? -> 账户安全
✅ 优化测试 4: 这个商品是正品吗? -> 商品咨询
✅ 优化测试 5: 微信支付失败了 -> 支付问题
✅ 优化测试 6: 需要开发票 -> 发票问题
✅ 优化测试 7: 我的银行卡绑定失败了 -> 支付问题
✅ 优化测试 8: 这个手机有什么颜色可以选择 -> 商品咨询
✅ 优化测试 9: 双十二有活动吗 -> 促销活动
✅ 优化测试 10: 电子发票怎么下载 -> 发票问题

📊 优化后准确率: 10/10 = 100.00%
📈 提升: 0.0%

=== 自定义测试 ===
🔍 我的银行卡绑定失败了 -> 支付问题
🔍 这个手机有什么颜色可以选择 -> 商品咨询
🔍 双十二有活动吗 -> 促销活动
🔍 电子发票怎么下载 -> 发票问题
🔍 账号密码忘记了 -> 账户安全
🔍 买的东西质量不好想退货 -> 售后服务
🔍 会员积分怎么使用 -> 促销活动
🔍 物流信息在哪里查看 -> 售后服务

🎉 测试完成！

📝 使用说明:
1. 确认API密钥格式: sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
2. 直接运行代码查看意图识别效果
3. 可以修改 custom_queries 列表测试更多场景
